In [2]:
from problem1 import *
from problem2 import *
from problem3 import *
import pandas as pd
import numpy as np

In [3]:
import pandas as pd

blob = pd.read_pickle(r'C:\MFE\MFE Sem 3\CSE 426\CSE_426\Project 2\data\blobs_data.pkl')
blob.keys()
model = SVMModel(blob['tr_X'], blob['tr_y'], 1, linear_kernel, 0.1)

In [ ]:
def train(model, max_iters = 10, record_every = 1, max_passes = 1, tol=1e-6):
    """
    SMO training of SVM
    model: an SVMModel
    max_iters: how many iterations of optimization
    record_every: record intermediate dual and primal objective values and models every record_every iterations
    max_passes: each iteration can have maximally max_passes without change any alpha, used in the SMO alpha selection.
    tol: numerical tolerance (exact equality of two floating numbers may be impossible).
    :return: 4 lists (of iteration numbers, dual objectives, primal objectives, and models)
    Hint: refer to subsection 3.5 "SMO" in notes.
    """
    #########################################
    # Creating the lists
    iter_num = []
    duals = []
    primals = []
    models = []

    # set dual variables to zero
    #alpha = np.zeros(model.train_y.shape)
    #b = 0

    # begin iteration
    for t in range(max_iters):
        passes = 0

        while passes < max_passes:
            num_changes = 0
            for i in range(model.m):
                
                if model.kernel_func.__name__ == 'linear_kernel':

                    kkt = model.train_y[0,i]*((np.dot(np.multiply(model.alpha, model.train_y), linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X).T) + model.b) - model.train_y[0,i])
                #    kkt= model.train_y[0,i]*((np.sum(np.multiply(model.alpha, model.train_y) @ linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X).T) + model.b) - model.train_y[0,i])
                #    kkt= model.train_y[0,i]*((np.sum((model.alpha* model.train_y) @ linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X).T) + model.b) - model.train_y[0,i])
                #    kkt = model.train_y[0,i]*(f_x(model.train_X, model.train_y, model.alpha, model.b, model.train_X[:,i], 1) - model.train_y[0,i])
                #    kkt = model.train_y[:,i]*((np.sum(np.multiply(model.alpha, model.train_y) * linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X).T) + model.b) - model.train_y[:,i])

                #    classifier = model.alpha[0,i]*model.train_y[0,i]*np.inner(model.train_X[0,i], model.train_X).sum() + model.b
                #    error = classifier - model.train_y[0,i]
                #    kkt = model.train_y[0,i]*error

                #    kkt = model.train_y[0,i]*(model.alpha[0,i]*np.inner(model.train_X[0,i], model.train_X).sum() + model.b) - 1
                #    kkt = model.train_y[0,i] - np.multiply(model.alpha, linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X)).sum() - model.b
                
                else:

                    kkt = model.train_y[0,i]*((np.dot(np.multiply(model.alpha, model.train_y), Gaussian_kernel(np.array([model.train_X[:,i]]), model.train_X).T) + model.b) - model.train_y[0,i])
                
                if (kkt < -1*tol and model.alpha[0,i] < model.C) or (kkt > tol and model.alpha[0,i] > 0):
                    # Random initialize where i != j               
                    list_for_randoms = [z for z in range(model.m)]

                    list_for_randoms.remove(i)
                    j = np.random.choice(list_for_randoms)
                    # Traing Example
                    x = np.array([model.train_X[:,i], model.train_X[:,j]]).T
                    alpha_vec = np.array([[model.alpha[0][i], model.alpha[0][j]]])
                    y_vec = np.array([[model.train_y[0][i], model.train_y[0][j]]])

                    # Upper and Lower Bounds
                    if y_vec[0,0] != y_vec[0,1]: 
                        H = min(model.C, model.C - alpha_vec[0,0] + alpha_vec[0,1])
                        L = max(0, -(alpha_vec[0,0] - alpha_vec[0,1]))

                    else:
                        H = min(model.C, alpha_vec[0,0] + alpha_vec[0,1])
                        L = max(0, alpha_vec[0,0] + alpha_vec[0,1] - model.C)


                    # Kernel Used
                    if model.kernel_func.__name__ == 'linear_kernel':
                        K = np.inner(x,x)
                    elif model.kernel_func.__name__ == 'Gaussian_kernel':
                        K = Gaussian_kernel(x, x)

                    #g vector with g_1 and g_2 inside for efficiency
                    g_vec = np.dot(np.multiply(alpha_vec, y_vec), K) + model.b

                    # alpha_2 value
                    #if K[0][0] + K[1][1] - 2*K[0][1] == 0:
                    #    continue
                    
                    alpha_new = alpha_vec[0, 1] + ((y_vec[0, 1]*(g_vec[0][0] - y_vec[0, 0] - g_vec[0][1] + y_vec[0, 1]))
                                            /(K[0][0] + K[1][1] - 2*K[0][1]))

                    #if alpha_new < 0:
                    #    alpha_new =0

                    # alpha_2 classification
                    if alpha_new > H:
                        alpha_new_clipped = H

                    elif alpha_new < L:
                        alpha_new_clipped = L

                    else: 
                        alpha_new_clipped = alpha_new

                    
                    # Stopping if not meaningful change
                    if np.abs(alpha_new_clipped-alpha_vec[0, 1]) < tol:

                        continue

                    # Getting alpha_1 if meaningful change
                    alpha_1_new = alpha_vec[0, 0] + (y_vec[0, 0]*y_vec[0, 1]) * (alpha_vec[0, 1] - alpha_new_clipped)

                    # b primal
                    # E vector (E_1, E_2)
                    E = g_vec - y_vec
                    b1 = model.b - E[0][0] - y_vec[0, 0]*(alpha_1_new - alpha_vec[0,0])*K[0][0] - y_vec[0, 1]*(alpha_new_clipped - alpha_vec[0, 1])*K[0][1]
                    b2 = model.b - E[0][1] - y_vec[0, 0]*(alpha_1_new - alpha_vec[0,0])*K[0][1] - y_vec[0, 1]*(alpha_new_clipped - alpha_vec[0, 1])*K[1][1]

                    # update model values
                    # alphas
                    model.alpha[0][i] = alpha_1_new
                    model.alpha[0][j] = alpha_new_clipped

                    # b primal
                    if 0 < model.alpha[0][i] < model.C:
                        b_new = b1
                    
                    elif 0 < model.alpha[0][j] < model.C:
                        b_new = b2

                    else:
                        b_new = (b1 + b2)/2

                    model.b = b_new
                    num_changes += 1

            if num_changes == 0:
                passes +=1
            else:
                passes = 0

        iter_num.append(t)
        duals.append(dual_objective_function(model.alpha, model.train_y, model.train_X, model.kernel_func, model.sigma))
        primals.append(primal_objective_function(model.alpha, model.train_y, model.train_X, model.b, model.C, model.kernel_func, model.sigma))
        models.append(vars(model))
    return iter_num, duals, primals, models

In [8]:
np.multiply(model.alpha,model.train_y).shape

(1, 100)

In [10]:
linear_kernel(model.train_X[:,i], model.train_X).shape

(100,)

In [11]:
np.dot(np.multiply(model.alpha,model.train_y), linear_kernel(model.train_X[:,i], model.train_X))

array([-32.78903542])

In [21]:
model = SVMModel(blob['tr_X'], blob['tr_y'], 1, linear_kernel, 0.1)
#model = SVMModel(blob['tr_X'], blob['tr_y'], 1, Gaussian_kernel, 0.1)

max_iters = 10 
record_every = 1 
max_passes = 1 
tol=1e-6
# Creating the lists
iter_num = []
duals = []
primals = []
models = []

# set dual variables to zero
#alpha = np.zeros(model.train_y.shape)
#b = 0

# begin iteration
for t in range(max_iters):
    passes = 0

    while passes < max_passes:
        num_changes = 0
        for i in range(model.m):
            
            if model.kernel_func.__name__ == 'linear_kernel':
            #    kkt= model.train_y[0,i]*((np.sum(np.multiply(model.alpha, model.train_y) @ linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X).T) + model.b) - model.train_y[0,i])

                #mu = model.C - model.alpha[0,i]
            #    wTx = 0
            #    for k in range(model.train_X.shape[1]):
            #        wTx += model.alpha[0,k]*model.train_y[0,k]*linear_kernel(model.train_X[:,k], model.train_X[:, i])
                #K = linear_kernel(model.train_X[:,i], model.train_X)
            #    kkt = sum(np.dot(np.multiply(model.alpha,model.train_y),K)) + model.b
                #kkt = model.alpha[0,i] * model.train_y[0,i] * sum(K) + model.b
                #kkt = model.train_y[0,i]*(np.dot(np.multiply(model.alpha,model.train_y),K)+model.b)
                #kkt = primal_objective_function(model.alpha, model.train_y, model.train_X, model.b, model.C, model.kernel_func, model.sigma)
                #error = max(0, 1-(kkt)) 
                kkt = 0
                for j in range(model.train_X.shape[1]):
                    kkt += model.train_y[0,j]*model.alpha[0,j]*linear_kernel(model.train_X[:,j], model.train_X[:,i]) 
                kkt = kkt - model.b
                kkt = model.train_y[0,i]*kkt    
            else:

                #kkt = model.train_y[0,i]*((np.dot(np.multiply(model.alpha, model.train_y), Gaussian_kernel(np.array([model.train_X[:,i]]), model.train_X).T) + model.b) - model.train_y[0,i])
                mu = model.C - model.alpha[0,i]
                wTx = 0
                for k in range(model.train_X.shape[1]):
                    wTx += model.alpha[0,k]*model.train_y[0,k]*Gaussian_kernel(np.array([model.train_X[:,k]]).T, np.array([model.train_X[:,i]]).T)

                kkt = ((wTx + model.b))
                error = max(0, 1-(wTx + model.b)) 
                    
            print(kkt)
            #print(model.alpha[0,i],kkt - 1 + error,model.alpha[0,i]*(kkt - 1 + error),mu,error,mu*error)
            #print(kkt, model.C,model.alpha[0,i])
            #if (kkt < -1*tol and model.alpha[0,i] < model.C) or (kkt > tol and model.alpha[0,i] > 0):
            #if ((kkt - 1 < 0) and  model.alpha[0,i] == 0) or (0 < model.alpha[0,i] < model.C and (kkt - 1 == tol or kkt - 1 == -tol) and error != 0) or (model.alpha[0,i] == model.C and (kkt - 1 + error != tol or kkt - 1 + error != -tol) and error >= 0):
            #if (model.alpha[0,i] < 0 or kkt - 1 + error < 0 or model.alpha[0,i]*(kkt - 1 + error) != 0 or mu < 0 or error < 0 or mu*error != 0):
            #if (((model.alpha[0,i] == 0 and kkt - 1 + error > 0 and error != 0) or (model.alpha[0,i] == 0 and kkt - 1 + error < 0 and error == 0) or (model.alpha[0,i] != 0 and kkt - 1 + error > 0 and error == 0)) or 
            #    ((0 < model.alpha[0,i] < model.C and kkt - 1 + error == 0 and error != 0) or (0 < model.alpha[0,i] < model.C and kkt - 1 + error != 0 and error == 0) or (0 > model.alpha[0,i] > model.C and kkt - 1 + error == 0 and error == 0)) or
            #    ((model.alpha[0,i] == model.C and kkt - 1 + error == 0 and error < 0) or (model.alpha[0,i] == model.C and kkt - 1 + error != 0 and error >= 0) or (model.alpha[0,i] != model.C and kkt - 1 + error == 0 and error >= 0))):
            #print((model.alpha[0,i] == 0 and  mu == model.C and error == 0 and kkt < 1 - error), (0 < model.alpha[0,i] < model.C and kkt != 1), (model.alpha[0,i] == model.C and kkt == 1 - error))
            #if ((model.alpha[0,i] == 0 and kkt < 1 and error==0) or 
            #    (0 < model.alpha[0,i] < model.C and  mu > 0 and error == 0 and kkt != 1 - error) or 
            #    (model.alpha[0,i] == model.C and mu == 0 and error >= 0 and kkt != 1 - error)):
            #if ((model.alpha[0,i] == 0 and mu ==  model.C and error == 0) or (model.alpha[0,i] == model.C and mu == 0 and error > 1)):
            #if (kkt < 1 and model.alpha[0,i] < model.C) or (kkt > 1 and model.alpha[0,i] > 0):
            if (model.alpha[0,i] == 0 and kkt < 1) or (0 < model.alpha[0,i] < model.C and kkt != 1) or (model.alpha[0,i] == model.C and kkt > 1):
                # Random initialize where i != j               
                list_for_randoms = [z for z in range(model.m)]

                list_for_randoms.remove(i)
                j = np.random.choice(list_for_randoms)

                # Traing Example
                x = np.array([model.train_X[:,i], model.train_X[:,j]]).T
                alpha_vec = np.array([[model.alpha[0][i], model.alpha[0][j]]])
                y_vec = np.array([[model.train_y[0][i], model.train_y[0][j]]])

                # Upper and Lower Bounds
                if y_vec[0,0] != y_vec[0,1]: 
                    H = min(model.C, model.C - alpha_vec[0,0] + alpha_vec[0,1])
                    L = max(0, alpha_vec[0,1] - alpha_vec[0,0])

                else:
                    H = min(model.C, alpha_vec[0,0] + alpha_vec[0,1])
                    L = max(0, alpha_vec[0,0] + alpha_vec[0,1] - model.C)


                # Kernel Used
                if model.kernel_func.__name__ == 'linear_kernel':
                    K = linear_kernel(x,x)
                elif model.kernel_func.__name__ == 'Gaussian_kernel':
                    K = Gaussian_kernel(x, x)
                #print(K)
                #g vector with g_1 and g_2 inside for efficiency
                g_vec = np.dot(np.multiply(alpha_vec, y_vec), K) + model.b

                # alpha_2 value
                #if K[0][0] + K[1][1] - 2*K[0][1] == 0:
                #    continue
                
                alpha_new = alpha_vec[0, 1] + ((y_vec[0, 1]*(g_vec[0][0] - y_vec[0, 0] - (g_vec[0][1] - y_vec[0, 1])))
                                        /(K[0][0] + K[1][1] - 2*K[0][1]))

                #if alpha_new < 0:
                #    alpha_new =0

                # alpha_2 classification
                if alpha_new > H:
                    alpha_new_clipped = H

                elif alpha_new < L:
                    alpha_new_clipped = L

                else: 
                    alpha_new_clipped = alpha_new

                
                # Stopping if not meaningful change
                if np.abs(alpha_new_clipped-alpha_vec[0, 1]) < tol:

                    continue

                # Getting alpha_1 if meaningful change
                alpha_1_new = alpha_vec[0, 0] + (y_vec[0, 0]*y_vec[0, 1]) * (alpha_vec[0, 1] - alpha_new_clipped)

                # b primal
                # E vector (E_1, E_2)
                E = g_vec - y_vec

                b1 = model.b - E[0][0] - y_vec[0, 0]*(alpha_1_new - alpha_vec[0,0])*K[0][0] - y_vec[0, 1]*(alpha_new_clipped - alpha_vec[0, 1])*K[0][1]
                b2 = model.b - E[0][1] - y_vec[0, 0]*(alpha_1_new - alpha_vec[0,0])*K[0][1] - y_vec[0, 1]*(alpha_new_clipped - alpha_vec[0, 1])*K[1][1]

                # update model values
                # alphas
                model.alpha[0][i] = alpha_1_new
                model.alpha[0][j] = alpha_new_clipped

                # b primal
                if 0 < model.alpha[0][i] < model.C:
                    b_new = b1
                
                elif 0 < model.alpha[0][j] < model.C:
                    b_new = b2

                else:
                    b_new = (b1 + b2)/2

                model.b = b_new
                num_changes += 1

        if num_changes == 0:
            passes +=1
        else:
            passes = 0

    iter_num.append(t)
    duals.append(dual_objective_function(model.alpha, model.train_y, model.train_X, model.kernel_func, model.sigma))
    primals.append(primal_objective_function(model.alpha, model.train_y, model.train_X, model.b, model.C, model.kernel_func, model.sigma))
    models.append(vars(model))



-0.0
0.0
0.7511521069585188
2.3362906763955964
1.6751689048152747
2.039696957471721
2.167248971320295
2.2182602083309635
2.0961358927499774
2.182181235496955
2.692190995522137
2.4052104715619715
2.4479727983724096
2.330722043692577
2.195012367815478
1.9485363808332994
2.018247126296489
1.678647994567373
2.0003428839085795
2.4931831209944377
2.0498644076657997
2.316915722006537
2.2716114874421107
2.093064180567831
1.8128321198286095
2.977130189310076
2.1706147103810034
2.0611730242749107
1.9095631414131602
2.4249618471792456
2.1478224046309697
1.692446408148503
2.651977170118581
1.794352380729293
1.7515478989606956
2.3318551596052846
2.4326986509311306
2.526093941809844
2.462486647860128
2.3235697988340096
2.2046593692912686
2.512679727681597
1.893355297639865
2.427547175346274
2.159029661067395
2.0854916428080523
2.440013248677534
2.269399504584027
1.8524384042899715
1.7993486124117557
1.967198673593118
2.516557683571615
2.0747388294168174
2.1432578275681107
2.270048320855831
2.1764308

KeyboardInterrupt: 

In [16]:
linear_kernel(model.train_X[:,j], model.train_X[:,i])

1.5466914615723717

In [19]:
model.alpha

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [242]:
model.alpha

array([[0.32688006, 0.        , 0.53626556, 0.06441392, 0.6509281 ,
        0.49873894, 0.30668074, 0.22660975, 0.27602648, 0.18586384,
        0.36218304, 0.12323028, 0.        , 0.23422726, 0.        ,
        0.        , 0.40771843, 0.17300349, 0.17555888, 0.        ,
        0.        , 0.26484021, 0.24642978, 0.22259095, 0.23719817,
        0.        , 0.23422726, 0.22361936, 0.37091904, 0.        ,
        0.24642978, 0.15227758, 0.30348847, 0.30668074, 0.        ,
        0.34640905, 0.19660585, 0.184737  , 0.        , 0.        ,
        0.        , 0.3837354 , 0.42399161, 0.22361936, 0.19636997,
        0.        , 0.21956105, 0.09702541, 0.27787307, 0.06057095,
        0.32836587, 0.42024835, 0.27602648, 0.19809073, 0.35358388,
        0.        , 0.03583002, 0.62265333, 0.23032464, 0.15423871,
        0.11355198, 0.21956105, 0.        , 0.        , 0.22259095,
        0.        , 0.03266274, 0.38800248, 0.32573545, 0.21316863,
        0.32688006, 0.23032464, 0.        , 0.18

In [244]:
model.alpha

array([[0.875 , 0.    , 0.375 , 0.75  , 0.9375, 0.5   , 0.    , 0.875 ,
        0.75  , 0.25  , 0.5   , 0.6875, 0.75  , 0.5625, 0.875 , 0.5   ,
        0.5   , 0.5   , 0.5   , 0.    , 0.    , 0.875 , 0.    , 0.5   ,
        0.75  , 0.75  , 0.875 , 0.6875, 0.5625, 0.5   , 0.5   , 0.375 ,
        0.    , 0.6875, 0.75  , 0.5   , 0.5   , 0.    , 0.75  , 0.875 ,
        0.9375, 0.    , 0.125 , 0.    , 0.25  , 0.75  , 0.5   , 0.25  ,
        0.75  , 0.75  , 0.5   , 0.5   , 0.75  , 0.875 , 0.6875, 0.875 ,
        0.5625, 0.875 , 0.75  , 0.25  , 0.5   , 0.5   , 0.5   , 0.    ,
        0.9375, 0.9375, 0.5   , 0.    , 0.5   , 0.6875, 0.5   , 0.    ,
        0.75  , 0.125 , 0.25  , 0.25  , 0.5   , 0.5   , 0.375 , 0.875 ,
        0.75  , 0.    , 0.5625, 0.875 , 0.5   , 0.25  , 0.6875, 0.5   ,
        0.5625, 0.75  , 0.875 , 0.25  , 0.5625, 0.75  , 0.75  , 0.875 ,
        0.5   , 0.5   , 0.75  , 0.75  ]])

In [ ]:
model.alpha

In [213]:
model = SVMModel(blob['tr_X'], blob['tr_y'], 1, linear_kernel, 0.1)
#model = SVMModel(blob['tr_X'], blob['tr_y'], 1, Gaussian_kernel, 0.1)
# Creating the lists
iter_num = []
duals = []
primals = []
models = []

# set dual variables to zero
#alpha = np.zeros(model.train_y.shape)
#b = 0

# begin iteration
for t in range(max_iters):
    passes = 0

    while passes < max_passes:
        num_changes = 0
        for i in range(model.m):
            
            if model.kernel_func.__name__ == 'linear_kernel':

            #    p = 0
            #    for k in range(model.train_X.shape[1]):
            #        p += (model.alpha[0,k]*model.train_y[0,k]*(linear_kernel(model.train_X[:,k], model.train_X[:,i])-1))
            #    p
            #    new = p + model.b
            #    final = new - model.train_y[0,i]
            #    kkt = model.train_y[0,i]*final

            #    kkt= model.train_y[0,i]*((np.sum(np.multiply(model.alpha, model.train_y) @ linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X).T) + model.b) - model.train_y[0,i])
            #    kkt= model.train_y[0,i]*((np.sum((model.alpha* model.train_y) @ linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X).T) + model.b) - model.train_y[0,i])
            #    kkt = model.train_y[0,i]*(f_x(model.train_X, model.train_y, model.alpha, model.b, model.train_X[:,i], 1) - model.train_y[0,i])
            #    kkt = model.train_y[:,i]*((np.sum(np.multiply(model.alpha, model.train_y) * linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X).T) + model.b) - model.train_y[:,i])

            #    classifier = model.alpha[0,i]*model.train_y[0,i]*np.inner(model.train_X[0,i], model.train_X).sum() + model.b
            #    error = classifier - model.train_y[0,i]
            #    kkt = model.train_y[0,i]*error

            #    kkt = model.train_y[0,i]*(model.alpha[0,i]*np.inner(model.train_X[0,i], model.train_X).sum() + model.b) - 1
            #    kkt = model.train_y[0,i] - np.multiply(model.alpha, linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X)).sum() - model.b
                mu = model.C - model.alpha[0,i]
                wTx = 0
                for k in range(model.train_X.shape[1]):
                    wTx += model.alpha[0,k]*model.train_y[0,k]*linear_kernel(model.train_X[:,k], model.train_X[:, i])

                kkt = (model.train_y[0,i]*(wTx + model.b))
                error = max(0, 1-model.train_y[0,i]*(wTx + model.b)) 
            else:

                #kkt = model.train_y[0,i]*((np.dot(np.multiply(model.alpha, model.train_y), Gaussian_kernel(np.array([model.train_X[:,i]]), model.train_X).T) + model.b) - model.train_y[0,i])
                mu = model.C - model.alpha[0,i]
                wTx = 0
                for k in range(model.train_X.shape[1]):
                    wTx += model.alpha[0,k]*model.train_y[0,k]*Gaussian_kernel(np.array([model.train_X[:,k]]).T, np.array([model.train_X[:,i]]).T)

                kkt = (model.train_y[0,i]*(wTx + model.b))
                error = max(0, 1-model.train_y[0,i]*(wTx + model.b)) 


            #print(kkt, model.C,model.alpha[0,i])
            #if (kkt < -1*tol and model.alpha[0,i] < model.C) or (kkt > tol and model.alpha[0,i] > 0):
            #if ((kkt - 1 < 0) and  model.alpha[0,i] == 0) or (0 < model.alpha[0,i] < model.C and (kkt - 1 == tol or kkt - 1 == -tol) and error != 0) or (model.alpha[0,i] == model.C and (kkt - 1 + error != tol) and error > 1):
            #if ((kkt - 1 < tol) and  model.alpha[0,i] == 0 and error == 0) or (0 < model.alpha[0,i] < model.C and (kkt - 1+error == tol or kkt - 1+error == -tol) and error <= 0) or (model.alpha[0,i] == model.C and (kkt - 1 + error != tol or kkt - 1 + error != -tol) and error >= 0):
            #if ((model.alpha[0,i] == 0 and mu == model.C and error == 0 and kkt < 1 - error) 
            #    or (0 < model.alpha[0,i] < model.C and mu > 0 and error == 0 and kkt != 1 - error)
            #    or (model.alpha[0,i] == model.C and mu == 0 and error >= 0 and kkt != 1 - error)):
            #if (model.alpha[0,i] < 0 or kkt - 1 + error < 0 or model.alpha[0,i]*(kkt - 1 + error) != 0 or mu < 0 or error < 0 or mu*error != 0):
                # Random initialize where i != j               
                list_for_randoms = [z for z in range(model.m)]

                list_for_randoms.remove(i)
                j = np.random.choice(list_for_randoms)
                # Traing Example
                x = np.array([model.train_X[:,i], model.train_X[:,j]]).T
                alpha_vec = np.array([[model.alpha[0][i], model.alpha[0][j]]])
                y_vec = np.array([[model.train_y[0][i], model.train_y[0][j]]])

                # Upper and Lower Bounds
                if y_vec[0,0] != y_vec[0,1]: 
                    H = min(model.C, model.C - alpha_vec[0,0] + alpha_vec[0,1])
                    L = max(0, alpha_vec[0,1] - alpha_vec[0,0])

                else:
                    H = min(model.C, alpha_vec[0,0] + alpha_vec[0,1])
                    L = max(0, alpha_vec[0,0] + alpha_vec[0,1] - model.C)


                # Kernel Used
                if model.kernel_func.__name__ == 'linear_kernel':
                    K = linear_kernel(x,x)
                elif model.kernel_func.__name__ == 'Gaussian_kernel':
                    K = Gaussian_kernel(x, x)
                #print(K)
                #g vector with g_1 and g_2 inside for efficiency
                g_vec = np.dot(np.multiply(alpha_vec, y_vec), K) + model.b

                # alpha_2 value
                if K[0][0] + K[1][1] - 2*K[0][1] == 0:
                    continue
                
                alpha_new = alpha_vec[0, 1] + ((y_vec[0, 1]*(g_vec[0][0] - y_vec[0, 0] - g_vec[0][1] + y_vec[0, 1]))
                                        /(K[0][0] + K[1][1] - 2*K[0][1]))

                #if alpha_new < 0:
                #    alpha_new =0

                # alpha_2 classification
                if alpha_new > H:
                    alpha_new_clipped = H

                elif alpha_new < L:
                    alpha_new_clipped = L

                else: 
                    alpha_new_clipped = alpha_new

                
                # Stopping if not meaningful change
                if np.abs(alpha_new_clipped-alpha_vec[0, 1]) < tol:

                    continue

                # Getting alpha_1 if meaningful change
                alpha_1_new = alpha_vec[0, 0] + (y_vec[0, 0]*y_vec[0, 1]) * (alpha_vec[0, 1] - alpha_new_clipped)

                # b primal
                # E vector (E_1, E_2)
                E = g_vec - y_vec

                b1 = model.b - E[0][0] - y_vec[0, 0]*(alpha_1_new - alpha_vec[0,0])*K[0][0] - y_vec[0, 1]*(alpha_new_clipped - alpha_vec[0, 1])*K[0][1]
                b2 = model.b - E[0][1] - y_vec[0, 0]*(alpha_1_new - alpha_vec[0,0])*K[0][1] - y_vec[0, 1]*(alpha_new_clipped - alpha_vec[0, 1])*K[1][1]

                # update model values
                # alphas
                model.alpha[0][i] = alpha_1_new
                model.alpha[0][j] = alpha_new_clipped

                # b primal
                if 0 < model.alpha[0][i] < model.C:
                    b_new = b1
                
                elif 0 < model.alpha[0][j] < model.C:
                    b_new = b2

                else:
                    b_new = (b1 + b2)/2

                model.b = b_new
                num_changes += 1

        if num_changes == 0:
            passes +=1
        else:
            passes = 0

    iter_num.append(t)
    duals.append(dual_objective_function(model.alpha, model.train_y, model.train_X, model.kernel_func, model.sigma))
    primals.append(primal_objective_function(model.alpha, model.train_y, model.train_X, model.b, model.C, model.kernel_func, model.sigma))
    models.append(vars(model))

KeyboardInterrupt: 

In [212]:
model.alpha

array([[1.        , 0.99999982, 0.99999904, 0.99999974, 0.99999898,
        1.        , 0.99999983, 0.99999889, 1.        , 1.        ,
        1.        , 1.        , 1.        , 0.99999905, 0.9999997 ,
        0.99999969, 0.99999812, 0.99999927, 1.        , 0.99999938,
        1.        , 1.        , 1.        , 1.        , 0.99999919,
        1.        , 1.        , 1.        , 1.        , 0.99999924,
        1.        , 1.        , 0.99999994, 0.99999926, 1.        ,
        0.99999899, 1.        , 1.        , 0.9999994 , 0.99999862,
        1.        , 1.        , 1.        , 0.9999989 , 1.        ,
        1.        , 1.        , 1.        , 1.        , 0.99999975,
        0.99999907, 1.        , 0.99999941, 0.9999998 , 0.99999984,
        0.99999957, 1.        , 1.        , 0.99999936, 0.99999969,
        0.99999936, 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        0.99999935, 1.        , 1.        , 0.99

In [214]:
model.alpha

array([[ 4.58627130e-01,  0.00000000e+00,  8.81712813e-01,
         2.80726391e-02,  1.00000000e+00,  0.00000000e+00,
         2.46272178e-01,  6.82344661e-01,  2.24059939e-01,
         2.19441955e-01,  0.00000000e+00,  0.00000000e+00,
         2.77555756e-17,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  4.97701259e-01,  2.61194520e-01,
         5.28589508e-01, -2.77555756e-17,  4.69438226e-01,
         0.00000000e+00,  0.00000000e+00,  1.35225400e-01,
         3.25480698e-02, -3.81639165e-17,  0.00000000e+00,
         5.55111512e-17,  5.26216627e-01,  0.00000000e+00,
         2.98020147e-01,  5.75104933e-01,  0.00000000e+00,
         1.00000000e+00,  4.43361692e-01,  0.00000000e+00,
         0.00000000e+00,  2.88724904e-01,  0.00000000e+00,
         2.87888296e-01,  3.80649568e-01,  0.00000000e+00,
         4.16950403e-01,  2.19441955e-01,  0.00000000e+00,
         3.48216722e-01,  1.40906209e-01,  1.15444125e-01,
         6.42919582e-01,  1.01887165e-01,  0.00000000e+0

In [142]:
from sklearn.datasets import make_blobs, make_circles, make_moons
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [143]:
n_samples = 100
X_train, y = make_blobs(n_samples, centers=2, n_features=2, random_state=1)

In [145]:
decision_function(model.alpha, model.train_y, model.train_X, model.b, linear_kernel, 0.1, X_train.T)

array([[ -31.31501645,  345.77554417,  -17.20566654,  -89.8476444 ,
         288.86100626,  324.87200702,  336.81848283,  341.37722053,
         -65.86106922,  -75.96386109,  386.2793888 ,  -96.06655362,
         363.81956998,  -88.00791515,  337.33407095,  316.20180113,
         323.66299518,  -27.86067192,  -58.03953211,  369.33057132,
         326.13539571,  351.92579037,  347.04279536,  -67.50274126,
         -40.67753598,  412.96581144,  337.4530987 ,  -64.53153497,
         -52.10399973,  360.83835557,  -71.76445505,  -28.51760237,
         383.21387371,  -39.21946158,  297.13725263,  353.19764407,
        -100.06628567,  372.74767999,  365.25571505,  352.36438824,
         340.81556808,  369.17098882,  -48.18673298,  363.4514196 ,
         -73.43186951,  329.47125005, -100.33865062,  -86.32135466,
         -43.32939644,  -39.77444286,  -55.14299553,  369.11077134,
         329.09060559,  336.06856814,  -83.58009305,  337.91727874,
         -67.03078481,  266.41324902,  -80.79513

In [140]:
model.alpha

array([[0.45979634, 0.36823187, 0.67692708, 0.13184387, 0.60190213,
        0.24744786, 0.38359493, 0.32302223, 0.22796015, 0.31131117,
        0.24152955, 0.16716928, 0.20640154, 0.21296454, 0.25926165,
        0.14053128, 0.20983017, 0.32494544, 0.38436204, 0.23200047,
        0.37362779, 0.24088098, 0.36507307, 0.18302267, 0.38925075,
        0.10709226, 0.38654494, 0.41025154, 0.09985915, 0.21169946,
        0.28025806, 0.32541488, 0.2012064 , 0.38004682, 0.31131117,
        0.23285707, 0.26105463, 0.30957003, 0.18793501, 0.43399686,
        0.17783531, 0.15253538, 0.28949209, 0.29756551, 0.13487547,
        0.11300307, 0.39328045, 0.33512226, 0.00889919, 0.04037955,
        0.22263791, 0.21280546, 0.17055081, 0.46724888, 0.31498398,
        0.1484487 , 0.4245637 , 0.37936157, 0.2012064 , 0.10284712,
        0.10884449, 0.44204638, 0.25539327, 0.24119721, 0.3128237 ,
        0.05828799, 0.06231872, 0.21376812, 0.20640154, 0.3806057 ,
        0.28025806, 0.11347886, 0.30125283, 0.25

In [114]:
model.alpha

array([[0.42672735, 0.34797334, 0.20485226, 0.31717897, 0.32423481,
        0.17339105, 0.24273077, 0.41834229, 0.20499182, 0.24936404,
        0.07353774, 0.13931277, 0.10401772, 0.30968182, 0.17572057,
        0.17429454, 0.34169653, 0.44120314, 0.2857535 , 0.15573808,
        0.36815267, 0.21450505, 0.29122968, 0.2474573 , 0.56766943,
        0.21814801, 0.11262778, 0.46883459, 0.39404682, 0.28546507,
        0.12250467, 0.3980395 , 0.10542531, 0.24273077, 0.2900291 ,
        0.12041324, 0.0443533 , 0.2105633 , 0.38846706, 0.32472015,
        0.24936404, 0.17989149, 0.36597907, 0.25996533, 0.36431825,
        0.28627378, 0.3238149 , 0.20282801, 0.35278332, 0.01510629,
        0.37132737, 0.17715481, 0.20485226, 0.20683247, 0.21747106,
        0.19313866, 0.39540924, 0.59345761, 0.03602317, 0.38035924,
        0.19089969, 0.23614408, 0.28133335, 0.22294567, 0.20627615,
        0.16743094, 0.23198105, 0.23580539, 0.36195711, 0.246091  ,
        0.358045  , 0.2474573 , 0.24082509, 0.09

In [99]:
model.alpha

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [50]:
wTx = 0
for k in range(model.train_X.shape[1]):
    wTx += model.alpha[0,k]*linear_kernel(model.train_X[0,k], model.train_X[0, i])

In [51]:
wTx

0.15424499511384682

In [55]:
model.alpha

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [65]:
model.train_X[:,3]

array([1.01140854, 1.14797295])

In [66]:
model.train_X[:,4]

array([-0.44558325, -1.02919521])

In [72]:
test = np.array([model.train_X[:,3],model.train_X[:,4]]).T
test

array([[ 1.01140854, -0.44558325],
       [ 1.14797295, -1.02919521]])

In [76]:
test.T

array([[ 1.01140854,  1.14797295],
       [-0.44558325, -1.02919521]])

In [83]:
sum(test.T[:,0]*test[1,:])

1.6196617995675748

In [84]:
test.T[:,0]

array([ 1.01140854, -0.44558325])

In [85]:
test[1,:]

array([ 1.14797295, -1.02919521])

In [73]:
np.inner(test, test)

array([[1.22149167, 1.6196618 ],
       [1.6196618 , 2.37708468]])

In [75]:
np.dot(test.T, test)

array([[ 2.34078913, -1.63215497],
       [-1.63215497,  1.25778722]])

In [50]:
X1.T

array([[1, 5],
       [7, 4]])

In [43]:
X2

array([[1, 7],
       [5, 4]])

In [52]:
X1 = np.array([[1, 5], [7, 4]]).T
X2 = np.array([[1, 5], [7, 4]]).T

np.inner(X1.T, X2.T)

array([[26, 27],
       [27, 65]])

In [36]:
X1 = np.array([[1, 2], [2, 4]]).T
X2 = np.array([[3, 4], [1, 2], [5, 6]]).T

In [38]:
X2

array([[3, 1, 5],
       [4, 2, 6]])

In [45]:
np.dot(X1.T, X2)

array([[26, 27],
       [27, 65]])

In [27]:
np.array([model.train_X[:,0]]).shape

(1, 2)

In [28]:
model.train_X.shape

(2, 100)

In [126]:
np.allclose(X1, X2)

True

In [119]:
X1.shape == X2.shape

True

In [107]:
np.dot(X1.T, X2)

array([[26, 27],
       [27, 65]])

In [93]:
np.dot(X1.T, X2)

array([[26, 27],
       [27, 65]])

In [90]:
X1 = np.array([[1, 2]]).T
X2 = np.array([[3, 4]]).T

np.dot(X1.T, X2)

array([[11]])

In [87]:
X1.T

array([[1, 5],
       [7, 4]])

In [115]:
np.dot(X1, X2)

array([[11,  5],
       [22, 10]])

In [47]:
np.array([[3, 4], [1, 2], [5, 6]])

array([[3, 4],
       [1, 2],
       [5, 6]])

In [41]:
np.dot(X1, X2)

array([[11,  5, 17],
       [22, 10, 34]])

In [130]:
X1 = np.array([[1, 2], [2, 4]]).T
X2 = np.array([[3, 4], [1, 2], [5, 6]] ).T

np.inner(X1, X2)

ValueError: shapes (2,2) and (3,2) not aligned: 2 (dim 1) != 3 (dim 0)

In [131]:
np.dot(X1.T, X2)

array([[11,  5, 17],
       [22, 10, 34]])

In [101]:
X1.T

array([[1, 2],
       [2, 4]])

In [103]:
X2

array([[3, 1],
       [4, 2]])

In [104]:
np.dot(X1.T, X2)

array([[11,  5],
       [22, 10]])

In [50]:
np.array([[3, 4], [1, 2]]).T

array([[3, 1],
       [4, 2]])

In [38]:
np.dot(X1.T, X2)

array([[11,  5],
       [22, 10]])

In [25]:
X1

array([[1, 2],
       [2, 4]])

In [26]:
X2

array([[3, 1],
       [4, 2]])

In [8]:
X1 = np.array([[1, 2]]).T
X2 = np.array([[3, 4]]).T
np.inner(X1, X2)

array([[3, 4],
       [6, 8]])

In [ ]:
np.dot()

In [6]:
X1 = np.array([[1, 2]]).T
X2 = np.array([[3, 4]]).T
Gaussian_kernel(X1, X2, 1)

array([[1.]])

In [12]:
np.exp(-(euclidean_distances(X1.T,X2.T)**2)/(2*1**2)).shape
X1.shape

(2, 1)